In [1]:
# WARNING - In Progress

import psycopg2
from datetime import timedelta
from sqlalchemy import create_engine
import pandas as pd
import numpy as np

import pickle
from sklearn.model_selection import StratifiedKFold, train_test_split
import scipy.io
import os
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from sklearn.metrics import roc_auc_score, average_precision_score, confusion_matrix, f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, GRU, LSTM, Conv2D, MaxPooling2D, Activation

import random
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K

import collections

import nest_asyncio
nest_asyncio.apply()

#%load_ext tensorboard

# CHANGE SQL FILE to flicu_pivoted_lab to only include ICU data

In [2]:
LOOKBACK = [5, 10, 24] # lookback (5h, 10h, 24h)
LOOKAHEAD = [3, 6, 12] # lookahead (3h, 6h, 12h)
MODEL = ['GRU', 'LSTM'] # model type ('GRU', 'LSTM')
CLIENTS = [2, 4, 8] # number of federated learning clients (2, 4, 8)
bs = 256 # batch size
lr = 0.01 # learning rate
comms_round = 50 # global epochs
model_loc='federated_models/' # folder for models to be saved in 
window_loc='windows_trial/' # folder to get windows files from

### Windowing 

In [3]:
import warnings
warnings.filterwarnings("ignore")#!!!!!!!!!!!!!!!!!!!!!!!

#-----
columns_pred = ['sysbp','diasbp','heartrate','tempc','resprate','wbc','ph','spo2','admission_age'] # parameters to use for the prediction task
columns_pred_idx = [x for x in range(len(columns_pred))]
#-----

In [4]:
def windows_load_process(file_name, lb):

    # load windows
    windows = pd.read_pickle(file_name)

    # sort dataframe by labels
    windows.sort_values(by=['label'])

    # split into X and y
    x_set_prime = np.array(windows['window'])
    x_set = np.zeros((len(x_set_prime),lb, len(columns_pred_idx)), float)
    for i in range(len(x_set_prime)):
        x_set[i] = x_set_prime[i]

    y_set = np.array(windows['label'])

    #print(x_set.shape)
    #print(y_set.shape)
    return x_set, y_set

### Normalize and Expand

In [5]:
# function for the normalization of the values based on the training data
# input : (data for standardisation parameter extraction, data to apply standardisation on)
def normalize(x_par,x_out):
    # one mean and std dev value for every parameter
    x_copy = x_par.copy()
    x_copy = x_copy.reshape(-1,len(columns_pred_idx)).transpose()
    
    means = np.zeros((len(columns_pred_idx)))
    stds = np.zeros((len(columns_pred_idx)))

    for i in range(0,len(x_copy)):
        means[i] = np.mean(x_copy[i])
        stds[i] = np.std(x_copy[i])

    # normalize the values
    x_out[:,:,:] -= means
    x_out[:,:,:] /= stds
    
    #transform from float64 to float32 format
    x_out = x_out.astype('float32')
    
    return x_out


In [6]:
# equalize the overall class proportions by duplicating the sepsis cases in the training set
def expand(x_train,y_train):
    """
    Info:
    =====
        - the function multiplies the sepsis cases in the training to improve the ANN training
    
    Parameter description:
    ======================
    
    x_train
    -------
        - the extracted features from the training set
        - x_train.shape[0] -> the samples
        - x_train.shape[1] -> the parameter values for each timestamp of the sequence (in case of sequence learning (RNN))
                           -> the higher level extracted features (in case of learning with extracted features)
        - x_train.shape[2] -> the parameters (in case of sequence learning)
                           -> not existent (in case of learning with extracted features)
    
    y_train
    -------
        - the labels of the corresponding samples
        - vector with the same length of x_train[0]
    """
    n = np.int((y_train.shape[0]-y_train.sum())/y_train.sum()) # duplicate n times
    if len(x_train.shape) == 3:
        sh = (np.int(n*y_train.sum()),x_train.shape[1],x_train.shape[2])
    else:
        sh = (np.int(n*y_train.sum()),x_train.shape[1])
    x_train_apdx = np.zeros(sh)
    x_train_sepsis = x_train[y_train>0]
    starti = 0
    endi = np.int(y_train.sum())
    for i in range(0,n):
        x_train_apdx[starti:endi] = x_train_sepsis
        starti = endi
        endi = np.int((i+2) * y_train.sum())
    x_train = np.concatenate((x_train,x_train_apdx),axis=0)
    y_train = np.concatenate((y_train,np.ones(x_train_apdx.shape[0])))

    return x_train, y_train

### RNN

In [7]:
def sens(y_true, y_pred):
    
    th = tf.constant(0.5, dtype=tf.float32)
    sv = tf.constant(1e-6, dtype=tf.float32)
    y_true = tf.cast(y_true, dtype=tf.bool)
    y_pred = y_pred > th # as the network outputs a value between [0,1] according to the sigmoid activation of the last unit
    
    y_true_ones = tf.boolean_mask(y_true, y_true)
    relating_y_pred = tf.boolean_mask(y_pred, y_true)

    res = tf.equal(y_true_ones, relating_y_pred)
    res = tf.cast(res, dtype=tf.float32)
    cM_11 = tf.reduce_sum(res) + sv # true positives cases (1e6 added to avoid division by 0 and therefor nan as result)
    cM_10 = tf.cast(tf.size(res), dtype=tf.float32) - cM_11 + sv # false negative cases (1e6 added to avoid division by 0 and therefor nan as result)
    sens = cM_11/(cM_11+cM_10)

    return sens


def spec(y_true, y_pred):
    
    th = tf.constant(0.5, dtype=tf.float32)
    sv = tf.constant(1e-6, dtype=tf.float32)
    y_true = tf.cast(y_true, dtype=tf.bool)
    y_pred = y_pred > th # as the network outputs a value between [0,1] according to the sigmoid activation of the last unit

    y_true_zeros = tf.boolean_mask(y_true, tf.logical_not(y_true))
    relating_y_pred = tf.boolean_mask(y_pred, tf.logical_not(y_true))
    y_true_zeros = tf.logical_not(y_true_zeros)
    relating_y_pred = tf.logical_not(relating_y_pred)

    res = tf.equal(y_true_zeros, relating_y_pred)
    res = tf.cast(res, dtype=tf.float32)
    cM_00 = tf.reduce_sum(res) + sv # true negative cases (1e6 added to avoid division by 0 and therefor nan as result)
    cM_01 = tf.cast(tf.size(res), dtype=tf.float32) - cM_00 + sv # false positive cases (1e6 added to avoid division by 0 and therefor nan as result)
    spec = cM_00/(cM_00+cM_01)
    
    return spec

def aurocSKL(y_true, y_pred, digits=3):
    
    res = roc_auc_score(y_true, y_pred) # function from scikitlearn (skl)
    res = round(res, digits)

    return res

def auprcSKL(y_true, y_pred, digits=3):
    
    res = average_precision_score(y_true, y_pred) # function from scikitlearn (skl)
    res = round(res, digits)

    return res

In [8]:
class GRU_MLP:
    @staticmethod
    def build(shape, classes):
        model = Sequential()
        model.add(GRU(40,recurrent_dropout=0.2,input_shape=(shape, classes),return_sequences=True))
        model.add(GRU(40,recurrent_dropout=0.2,input_shape=(shape, classes)))
        # model.add(Dropout(0.5))
        model.add(Dense(1,activation='sigmoid'))
        return model

class LSTM_MLP:
    @staticmethod
    def build(shape, classes):
        model = Sequential()
        model.add(LSTM(40,recurrent_dropout=0.2,input_shape=(shape, classes),return_sequences=True))
        model.add(LSTM(40,recurrent_dropout=0.2,input_shape=(shape, classes)))
        # model.add(Dropout(0.5))
        model.add(Dense(1,activation='sigmoid'))
        return model


In [24]:
def weight_scalling_factor(clients_trn_data, client_name):
    client_names = list(clients_trn_data.keys())
    #get the bs
    bs = list(clients_trn_data[client_name])[0][0].shape[0]
    #first calculate the total training data points across clinets
    global_count = sum([tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy() for client_name in client_names])*bs
    # get the total number of data points held by a client
    local_count = tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy()*bs
    return local_count/global_count


def scale_model_weights(weight, scalar):
    '''function for scaling a models weights'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar * weight[i])
    return weight_final



def sum_scaled_weights(scaled_weight_list):
    '''Return the sum of the listed scaled weights. The is equivalent to scaled avg of the weights'''
    avg_grad = list()
    #get the average grad accross all client gradients
    for grad_list_tuple in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)
        
    return avg_grad


def test_model(X_test, Y_test, model, fold, verbose=1):
    Y_test = tf.expand_dims(Y_test, 1)
    cce = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    #logits = model.predict(X_test, batch_size=100)
    logits = model.predict(X_test)
    loss = cce(Y_test, logits)
    acc = accuracy_score(Y_test,(logits > 0.5).astype(np.float32))
    auroc = aurocSKL(Y_test,logits)
    auprc = auprcSKL(Y_test,logits)
    f1 = f1_score(Y_test,(logits > 0.5).astype(np.float32))
    if verbose == 1:
        print('FOLD: {} | global_acc: {:.3%} | global_loss: {:.3%} | global_auroc: {:.3%} | global_auprc: {:.3%} | global_f1: {:.3%}'.format(fold, acc, loss, auroc, auprc, f1))
    return acc, loss, auroc, auprc, f1

### Federated Members (clients) as Data Shards

In [10]:
def create_clients(train_list, label_list, bs=32, num_clients=3, initial='clients'):
    ''' return: a dictionary with keys clients' names and value as 
                data shards - tuple of images and label lists.
        args: 
            image_list: a list of numpy arrays of training images
            label_list:a list of binarized labels for each image
            num_client: number of fedrated members (clients)
            initials: the clients'name prefix, e.g, clients_1 
            
    '''

    #create a list of client names
    client_names = ['{}_{}'.format(initial, i+1) for i in range(num_clients)]

    #randomize the data
    data = list(zip(train_list, label_list))
    random.shuffle(data)

    #shard data and place at each client
    size = len(data)//num_clients
    shards = [data[i:i + size] for i in range(0, size*num_clients, size)]

    shards_train = []
    shards_val = []
    # normalize and expand shards
    for shard in shards:
        # unzip train data and labels
        x_train_c = np.array([i.tolist() for i, j in shard])
        y_train_c = np.array([j.tolist() for i, j in shard])

        # split shard into training and validation sets
        x_train_c, x_val_c, y_train_c, y_val_c = train_test_split(x_train_c,y_train_c,test_size=0.25,stratify=y_train_c,random_state=1)
        
        # standardise training and validation data
        x_train_c = normalize(x_train_c,x_train_c)
        x_val_c = normalize(x_train_c,x_val_c)

        # expand training data
        x_train_c, y_train_c = expand(x_train_c,y_train_c)

        # process and batch the training data for each client
        shards_train.append(tf.data.Dataset.from_tensor_slices((x_train_c, y_train_c)).shuffle(len(y_train_c)).batch(bs))
        shards_val.append(tf.data.Dataset.from_tensor_slices((x_val_c, y_val_c)).batch(len(y_val_c)))

    #number of clients must equal number of shards
    assert(len(shards_train) == len(shards_val) == len(client_names))

    clients_train = {client_names[i] : shards_train[i] for i in range(len(client_names))}
    clients_val = {client_names[i] : shards_val[i] for i in range(len(client_names))}

    return clients_train, clients_val

### Process

In [11]:
loss='binary_crossentropy'
metrics = ['accuracy', sens, spec]
optimizer = SGD(lr=lr, 
                decay=lr / comms_round, 
                momentum=0.9
               )  

1. 4-fold stratified cross-validation
2. split in the ratio of 9:3:4
3. batch size of 32
4. look-back window lengths (5h, 10h, 24h) ~
5. prediction times (3h, 6h, 12h) ~
6. FL clients are 2, 4, and 8 ~
7. learning rate of 0.01
8. 50 FL rounds
9. local model 3 epochs

In [12]:
def process(x_set, y_set, lb, pt, model_type, client_cnt, nsplits, comms_round=30, verbose_round=0, verbose_fold=0):

    label_encoder = LabelEncoder()
    y_set = label_encoder.fit_transform(y_set)

    # generate stratified folds for crossvalidation
    skf = StratifiedKFold(n_splits=nsplits,shuffle=True,random_state=1) # specify random_state to produce the same random numbers each time

    # initialize stratified kFold generator
    splits = skf.split(x_set,y_set)

    split_c = -1

    fold_cnt = 0

    model_history = []
    for train_idx, test_idx in splits:
        fold_cnt += 1
        split_c += 1
        x_train = x_set[train_idx]
        y_train = y_set[train_idx]
        x_test = x_set[test_idx]
        y_test = y_set[test_idx]

        # normalize testing data based on global training set
        x_test = normalize(x_train,x_test)

        #print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

        #create, process and batch clients
        clients_train, clients_val = create_clients(x_train, y_train, bs, num_clients=client_cnt, initial='client')

        #process and batch the test set  
        test_batched = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(len(y_test))
        
        
        #initialize global model
        smlp_global = GRU_MLP()
        if model_type == 'LSTM':
            smlp_global = LSTM_MLP()
        global_model = smlp_global.build(x_train.shape[1], x_train.shape[2])
                
        #commence global training loop
        global_val_metrics = []
        for comm_round in range(comms_round):
                    
            # get the global model's weights - will serve as the initial weights for all local models
            global_weights = global_model.get_weights()
            
            #initial list to collect local model weights after scalling
            scaled_local_weight_list = list()

            #randomize client data - using keys
            client_names= list(clients_train.keys())
            random.shuffle(client_names)
            
            #loop through each client and create new local model
            local_metrics = []
            for client in client_names:
                smlp_local = GRU_MLP()
                if model_type == 'LSTM':
                    smlp_local = LSTM_MLP()
                local_model = smlp_local.build(x_train.shape[1], x_train.shape[2])
                local_model.compile(loss=loss, 
                            optimizer=optimizer, 
                            metrics=metrics)
                
                #set local model weight to the weight of the global model
                local_model.set_weights(global_weights)
                
                #fit local model with client's data
                local_model.fit(clients_train[client], epochs=3, verbose=0)
                
                #scale the model weights and add to list
                scaling_factor = weight_scalling_factor(clients_train, client)
                scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
                scaled_local_weight_list.append(scaled_weights)
                #test local model
                for(X_val, Y_val) in clients_val[client]:
                    local_acc, local_loss, local_auroc, local_auprc, local_f1 = test_model(X_val, Y_val, local_model, comm_round, verbose=0)
                    local_metrics.append([local_acc, local_loss, local_auroc, local_auprc, local_f1])
                
            #to get the average over all the local model, we simply take the sum of the scaled weights
            average_weights = sum_scaled_weights(scaled_local_weight_list)
            
            #update global model 
            global_model.set_weights(average_weights)

            # transpose local metrics and average
            local_metrics = np.array(local_metrics).T.tolist()
            local_metrics = [sum(r)/len(r) for r in local_metrics]

            if verbose_round == 1:
                print('round: {} | global_val_acc: {:.3%} | global_val_loss: {:.3%} | global_val_auroc: {:.3%} | global_val_auprc: {:.3%} | global_val_f1_score: {:.3%}'.format(comm_round, local_metrics[0], local_metrics[1], local_metrics[2], local_metrics[3], local_metrics[4]))
            
            #clear session to free memory after each communication round
            K.clear_session()
                
            # stop criteria (on AUROC)
            global_val_metrics.append(local_metrics)
            if len(global_val_metrics) >= 21:
                if global_val_metrics[comm_round-20][1] > global_val_metrics[-1][1]: # check for stagnation/overfitting
                    print('Stopped early: round ', comm_round)
                    break

        #test global model and print out metrics for current fold
        global_metrics = []
        for(X_test, Y_test) in test_batched:
            global_acc, global_loss, global_auroc, global_auprc, global_f1 = test_model(X_test, Y_test, global_model, fold_cnt, verbose=verbose_fold)
            global_metrics = [global_acc, global_loss, global_auroc, global_auprc, global_f1]

        model_history.append([global_metrics[0], global_metrics[1], global_metrics[2], global_metrics[3], global_metrics[4], global_model])

    # identify best model (auroc) across folds
    # format: model, lookback, lookahead, clients, accuracy, loss, auroc, auprc
    best_idx = np.array(model_history).T.tolist()[2].index(max(np.array(model_history).T.tolist()[2]))
    best = model_history[best_idx]
    
    # save model
    fname = str(model_type)+'_Model_lookback_'+str(lb)+'_lookahead_'+str(pt)+'_flClients_'+str(client_cnt)+'_aurocTest_'+str(best[2])+'_auprcTest_'+str(best[3])+'_f1score_'+str(best[4])+'.h5'
    best[4].save(model_loc+fname)   

    cross_val_best.append([str(model_type), lb, pt, client_cnt, best[0], best[1], best[2], best[3], best[4]])

In [25]:
#-------------
nsplits = 2
verbose_round = 0
verbose_fold = 1
#-------------

cross_val_best = [['model', 'lookback', 'lookahead', 'clients', 'accuracy', 'loss', 'auroc', 'auprc', 'f1_score']]

for lb in LOOKBACK:
    for pt in LOOKAHEAD:

        # load and process windows
        windows_file = window_loc+'Windows_lookback_'+str(lb)+'_lookahead_'+str(pt)+'.pkl'
        x_set, y_set = windows_load_process(windows_file, lb)

        for client_cnt in CLIENTS:

            for model_type in MODEL:

                print('---STARTING--- model:'+model_type+' lookback:'+str(lb)+' lookahead:'+str(pt)+' clients:'+str(client_cnt)+' -----')
                # train and test
                best_model_par = process(x_set, y_set, lb, pt, model_type, client_cnt, nsplits, comms_round, verbose_round, verbose_fold)
                cross_val_best.append(best_model_par)
                print('---------------------------------------------------------------------------')

print(cross_val_best)

---STARTING--- model:GRU lookback:5 lookahead:3 clients:2 -----
